In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
grid = []
row_limits = []
col_limits = []

instructions = []

with open("./data/day22.txt") as f:
    grid_data, instructions_data = f.read().split('\n\n')

for row in grid_data.split('\n'):
    row = row.rstrip()
    min_col, max_col = row.count(' '), len(row)-1
    grid.append(row)
    row_limits.append((min_col, max_col))

for col_ind in range(max(map(len, grid))):
    col = ''.join([grid[i][col_ind] if col_ind < len(grid[i]) else ' ' for i in range(len(grid))]).rstrip()
    min_row, max_row = col.count(' '), len(col)-1
    col_limits.append((min_row, max_row))

instructions_data = instructions_data.replace('R', ' R ').replace('L', ' L ').split(' ')
for inst in instructions_data:
    try:
        instructions.append(int(inst))
    except ValueError:
        instructions.append(inst)

In [3]:
def move(position, direction, count):
    if direction[0] != 0:
        limits = (col_limits[position[1]][0], col_limits[position[1]][1])
    else:
        limits = (row_limits[position[0]][0], row_limits[position[0]][1])
    for _ in range(count):
        if direction[0] != 0:
            next_position = [position[0]+direction[0], position[1]]
            if next_position[0] < limits[0]:
                next_position[0] = limits[1]
            elif next_position[0] > limits[1]:
                next_position[0] = limits[0]
        else:
            next_position = [position[0], position[1]+direction[1]]
            if next_position[1] < limits[0]:
                next_position[1] = limits[1]
            elif next_position[1] > limits[1]:
                next_position[1] = limits[0]
        if grid[next_position[0]][next_position[1]] == '#':
            break
        position = next_position
    return position

In [4]:
def turn(direction, char):
    if char == 'R':
        return [direction[1], -direction[0]]
    return [-direction[1], direction[0]]

In [5]:
position = [0, row_limits[0][0]]
direction = [0, 1]

for inst in instructions:
    if isinstance(inst, int):
        position = move(position, direction, inst)
    else:
        direction = turn(direction, inst)

position, direction

([42, 115], [0, -1])

In [6]:
part1 = (position[0]+1)*1000 + (position[1]+1)*4 + (0 if direction == [0, 1]
                                                    else 1 if direction == [1, 0]
                                                    else 2 if direction == [0, -1]
                                                    else 3)
part1

43466

# Part 2

I guess the shape of the cube might not be the same for everyone since at least mine is different than the example. Here is my shape:

In [7]:
#      .1......2.
#      6........3
#      ..........
#      ..........
#      ........4.
#      .....
#      7....
#      .....
#      ....4
#      .....
# .7........
# ..........
# ..........
# 6........3
# ........5.
# .....
# 1....
# .....
# ....5
# ...2.

The numbers indicate exactly how the sides are connected. I'll just write a function that has these connections hardcoded to compute the new position whenever we need to wrap around the cube.

In [8]:
def wrap(position, direction):
    row, col = position
    if row == -1 and col in range(50, 100) and direction == [-1, 0]: # 1
        new_row = 100+col
        new_col = 0
        new_direction = [0, 1]
    elif row == -1 and col in range(100, 150) and direction == [-1, 0]: # 2
        new_row = 199
        new_col = col-100
        new_direction = [-1, 0]
    elif row in range(0, 50) and col == 150 and direction == [0, 1]: # 3
        new_row = 149-row
        new_col = 99
        new_direction = [0, -1]
    elif row == 50 and col in range(100, 150) and direction == [1, 0]: # 4
        new_row = 99-(149-col)
        new_col = 99
        new_direction = [0, -1]
    elif row in range(50, 100) and col == 100 and direction == [0, 1]: # 4
        new_row = 49
        new_col = 149-(99-row)
        new_direction = [-1, 0]
    elif row in range(100, 150) and col == 100 and direction == [0, 1]: # 3
        new_row = 149-row
        new_col = 149
        new_direction = [0, -1]
    elif row == 150 and col in range(50, 100) and direction == [1, 0]: # 5
        new_row = 199-(99-col)
        new_col = 49
        new_direction = [0, -1]
    elif row in range(150, 200) and col == 50 and direction == [0, 1]: # 5
        new_row = 149
        new_col = 99-(199-row)
        new_direction = [-1, 0]
    elif row == 200 and col in range(0, 50) and direction == [1, 0]: # 2
        new_row = 0
        new_col = 100+col
        new_direction = [1, 0]
    elif row in range(150, 200) and col == -1 and direction == [0, -1]: # 1
        new_row = 0
        new_col = row-100
        new_direction = [1, 0]
    elif row in range(100, 150) and col == -1 and direction == [0, -1]: # 6
        new_row = 149-row
        new_col = 50
        new_direction = [0, 1]
    elif row == 99 and col in range(0, 50) and direction == [-1, 0]: # 7
        new_row = 50+col
        new_col = 50
        new_direction = [0, 1]
    elif row in range(50, 100) and col == 49 and direction == [0, -1]: # 7
        new_row = 100
        new_col = row-50
        new_direction = [1, 0]
    elif row in range(0, 50) and col == 49 and direction == [0, -1]: # 6
        new_row = 149-row
        new_col = 0
        new_direction = [0, 1]
    else:
        print(position, direction)
        raise Exception('Off the grid in an unexpected way.')
    return [new_row, new_col], new_direction

In [9]:
def move2(position, direction, count):
    for _ in range(count):
        next_direction = direction
        
        if direction[0] != 0:
            limits = (col_limits[position[1]][0], col_limits[position[1]][1])
        else:
            limits = (row_limits[position[0]][0], row_limits[position[0]][1])

        if direction[0] != 0:
            next_position = [position[0]+direction[0], position[1]]
            if next_position[0] < limits[0]:
                next_position, next_direction = wrap(next_position, direction)
            elif next_position[0] > limits[1]:
                next_position, next_direction = wrap(next_position, direction)
        else:
            next_position = [position[0], position[1]+direction[1]]
            if next_position[1] < limits[0]:
                next_position, next_direction = wrap(next_position, direction)
            elif next_position[1] > limits[1]:
                next_position, next_direction = wrap(next_position, direction)

        if grid[next_position[0]][next_position[1]] == '#':
            break

        position = next_position
        direction = next_direction

    return position, direction

In [10]:
position = [0, row_limits[0][0]]
direction = [0, 1]

for inst in instructions:
    if isinstance(inst, int):
        position, direction = move2(position, direction, inst)
    else:
        direction = turn(direction, inst)

position, direction

([161, 37], [-1, 0])

In [11]:
part2 = (position[0]+1)*1000 + (position[1]+1)*4 + (0 if direction == [0, 1]
                                                    else 1 if direction == [1, 0]
                                                    else 2 if direction == [0, -1]
                                                    else 3)
part2

162155